# INDIEMART ONLINE RETAIL ANALYSIS
![pngegg.png](pngegg.png)

## Import Libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import scipy
import re
from sqlalchemy import create_engine
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

conn = sqlite3.connect('C:/Users/Azriel Akbar/Documents/Courses/Personal Project/Indiemart Challenge/Indiemart dbeaver/indiemart.db')

## Load Data from Database

In [ ]:
# Extract price data from database
query = "SELECT * FROM prices"
price = pd.read_sql_query(query, conn)
price.head()

In [ ]:
price['created_at'].nunique()

In [ ]:
# Extract items data from database
query = "SELECT * FROM items"
item = pd.read_sql_query(query, conn)
item.head()

## Indiemart Dataset

In [ ]:
# Left join the price table and item table
price_item = price.merge(item, left_on='items_id', right_on='id', how='left')

In [ ]:
# Drop least needed columns
price_item.drop(columns=['id_y','description','image','link','created_at_y'], inplace=True)

In [ ]:
price_item.head()

In [ ]:
price_item = price_item[price_item['price']!=0]

In [ ]:
price_item.rename(columns={'id_x':'id','created_at_x':'order_time'}, inplace=True)

In [ ]:
# Get basic info of columns
price_item.info()

In [ ]:
price_item.isna().any()

In [ ]:
price_item[price_item.duplicated()]

In [ ]:
price_item['order_time'] = pd.to_datetime(price_item['order_time'])

In [ ]:
price_item['category'] = price_item['category'].str.title()
price_item['category'] = price_item['category'].str.replace(' Dan ', ' & ')
price_item['category'] = price_item['category'].str.replace('  ', ' ')
price_item['name'] = price_item['name'].str.rstrip()
price_item['name'] = price_item['name'].str.lstrip()

In [ ]:
price_item.nunique()

In [ ]:
price_item['date'] = price_item['order_time'].dt.date
price_item['timestamp'] = price_item['order_time'].dt.time
price_item['dow'] = price_item['order_time'].dt.strftime('%A')

In [ ]:
price_item.sample(5)

In [ ]:
indo_cat = list(price_item.query('source == "klikindomaret"')['category'].unique())

In [ ]:
alfa_cat = list(price_item.query('source == "alfagift"')['category'].unique())

In [ ]:
# uniform-ing categories between sources
conditions = [
    price_item['category'].isin(['Perlengkapan Anak', 'Perlengkapan Bayi', 'Perlengkapan Menyusui', 'Popok Bayi']),
    price_item['category'].isin(['Gas Elpiji', 'Handuk', 'Bed Cover & Sprei', 'Household Non Electric Hobby Product',
                                  'Pembasmi Kuman & Pewangi', 'Deterjen & Pembersih', 'Elektronik Rumah Tangga']),
    price_item['category'].isin(['Sarapan', 'Makanan Instan', 'Cokelat Permen', 'Cemilan Biskuit',
                                  'Buah & Dessert', 'Makanan Siap Saji', 'Makanan Kaleng', 'Oriental Food',
                                  'Western Food', 'Korean Food', 'Makanan Bayi', 'Aneka Roti', 'Healthy Food']),
    price_item['category'].isin(['Jus', 'Kental Manis', 'Susu', 'Kopi', 'Teh', 'Air Mineral', 'Es Krim',
                                  'Sirup', 'Minuman Tradisional', 'Coklat Bubuk Kremer','Minuman Ringan']),
    price_item['category'].isin(['Sayur', 'Buah & Dessert', 'Makanan Beku','Dagingayam Seafood','Olahan Susu Telur']),
    price_item['category'].isin(['Cosmetic', 'Hair Care', 'Body Care', 'Perawatan Diri',
                                  'Mens Care', 'Produk Tisu']),
    price_item['category'].isin(['Health Care','Alat Kontrasepsi']),
    price_item['category'].isin(['Mainan & Hobi', 'Tas Travel', 'Florist', 'Kartu Perdana', 'Kartu Giftcard',
                                  'Gadget Accessories', 'Mens Fashion', 'Womens Fashion','Jas Hujan', 'Kids Fashion',
                                  'Stationery Special Items','Mekanik Elektrik','Rokok','Computer 2']),
    price_item['category'] == 'Perlengkapan Makanan Hewan',
    price_item['category'].isin(['Bahan Makanan', 'Bahan Kue'])
]

choices = [
    'Kebutuhan Ibu & Anak',
    'Kebutuhan Rumah',
    'Makanan',
    'Minuman',
    'Produk Segar & Beku',
    'Personal Care',
    'Kebutuhan Kesehatan',
    'Lifestyle',
    'Pet Foods',
    'Kebutuhan Dapur'
]

price_item['grand_category'] = np.select(conditions, choices, default=price_item['category'])

In [ ]:
price_item.head()

In [ ]:
price_item.nunique()

In [ ]:
price_item['dow'].unique()

## Descriptive Analysis

### 1. Revenue comparison between sources

In [ ]:
fig = px.histogram(price_item, x='grand_category', y='price', color='source', barmode='group', height=400)

fig.update_layout(title='Revenue Comparison',
                  xaxis=dict(title='Category'),
                  yaxis=dict(title='IDR (billions)')
                 )
fig.show()

- **Insight:** from the diagram above, we can see that `alfagift` were generate more revenue, with *Kebutuhan Ibu & Anak and Personal Care* categories as the 2-top contributors. While, `klikindomaret`'s revenue, mainly comes from *Makanan, Lifestyle, and Personal Care.*

### 2. Comparison of average prices on each categories

In [ ]:
avg_price_by_category = price_item.groupby(['source','grand_category'])['price'].mean().reset_index()

fig = px.scatter(avg_price_by_category, x='price', y='grand_category', color='source', title="Dumbbell Chart: Average Price Comparison on each Source's Categories")

for i in avg_price_by_category['grand_category'].unique():
    cat_sub = avg_price_by_category[avg_price_by_category['grand_category'] == i]
    
    fig.add_shape(
    type='line',
    layer='below',
    y0=cat_sub.grand_category.values[0], x0=cat_sub.price.values[0],
    y1=cat_sub.grand_category.values[1], x1=cat_sub.price.values[1])

fig.show()

### 3. Product with the most sold and the least sold

#### Top Sold Product

In [ ]:
topleast30_sold = price_item.groupby(['name','source'])['id'].agg('count').reset_index().sort_values(by='id', ascending=False)
top30_sold = topleast30_sold.head(30)
least30_sold = topleast30_sold.tail(30)

In [ ]:
fig = px.bar(top30_sold, x='name', y='id', color='source', height=700)

fig.update_layout(title='Top Sold Product',
                  xaxis=dict(title='Product'),
                  yaxis=dict(title='Count')
                 )
#fig.update_traces(marker=dict(color=top30_sold['id'].tolist()))
fig.show()

#### Least Sold Product

In [ ]:
fig = px.bar(least30_sold, x='name', y='id', color='source', height=700)

fig.update_layout(title='Least Sold Product',
                  xaxis=dict(title='Product'),
                  yaxis=dict(title='Count')
                 )
fig.show()

### 4. Purchase time heatmap

In [ ]:
order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
order_heatmap = price_item.pivot_table(index='dow', columns=price_item['order_time'].dt.floor('30min').dt.time,\
                                                values='id', aggfunc='count').reset_index()
order_heatmap.fillna(0, inplace=True)
order_heatmap = order_heatmap.set_index('dow').reindex(order)

In [ ]:
# purchase_heat = price_item.groupby([pd.Categorical(price_item['dow'], categories= order, ordered=True),\
#                                     price_item['order_time'].dt.floor('30min').dt.time])['id'].agg('count').reset_index()
# purchase_heat.rename(columns={'level_0':'dow'}, inplace=True)
# purchase_heat

In [ ]:
plt.figure(figsize=(18,8))
sns.heatmap(order_heatmap, annot=True, fmt='0.0f', cmap='crest')

plt.title('Purchase Time Heatmap')
plt.xlabel('Hours')
plt.ylabel('Day of Week')

plt.show()

## Save Processed Data

In [ ]:
# save table to parquet
price_item.to_csv('C:/Users/Azriel Akbar/Documents/Courses/Personal Project/Indiemart Challenge/price_item.csv')
order_heatmap.to_csv('C:/Users/Azriel Akbar/Documents/Courses/Personal Project/Indiemart Challenge/order_heatmap.csv')